In [1]:
!pip install python-binance
!pip install ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 65 kB 577 kB/s 
     |████████████████████████████████| 1.1 MB 11.3 MB/s 
     |████████████████████████████████| 288 kB 59.0 MB/s 
     |████████████████████████████████| 112 kB 51.9 MB/s 
     |████████████████████████████████| 45 kB 2.2 MB/s 
     |████████████████████████████████| 94 kB 3.1 MB/s 
     |████████████████████████████████| 144 kB 52.4 MB/s 
     |████████████████████████████████| 271 kB 49.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for ta: filename=ta-0.10.1-py3-none-any.whl size=28985 sha256=f274a3d09dfb72f503327a28d178a510c6ceefcf8779f145721e6ec3e8c1f5d1
  Stored in directory: /root/.cache/pip/wheels/bc/2a/c2/a56e77d07edc16a1fa7fb012667e55cb0643cfa65996bddecc
Successfully built ta


In [2]:
from binance import Client

In [3]:
import pandas as pd
import numpy as np
import ta
import time

In [4]:

client = Client(api_key,api_secret)

In [5]:
def pulldata(symbol,interval,lookback):
  dataframe = pd.DataFrame(client.get_historical_klines(symbol,interval,lookback + 'min ago UTC'))
  dataframe = dataframe.iloc[:,:6]
  dataframe.columns = ['Time','Open','High','Low','Close','Volume']
  dataframe = dataframe.set_index('Time')
  dataframe.index = pd.to_datetime(dataframe.index, unit = 'ms')
  dataframe = dataframe.astype(float)
  return dataframe

In [137]:
df = pulldata('BTCUSDT', '1m','10000')

In [143]:
def applytechnicals(df):
  df['%K'] = ta.momentum.stoch(df.High,df.Low,df.Close,window=14,smooth_window=3)
  df['%D'] = df['%K'].rolling(3).mean()
  df['rsi'] = ta.momentum.rsi(df.Close, window = 14)
  df['macd'] = ta.trend.macd_diff(df.Close)
  df.dropna(inplace=True)
  return df


In [144]:
applytechnicals(df)

,Open,High,Low,Close,Volume,%K,%D,rsi,macd,trigger,Buy
Time,,,,,,,,,,,
2022-05-27 06:06:00,28699.81,28709.97,28684.24,28684.24,35.89506,5.079276,10.260178,32.175445,-5.660928,1,0
2022-05-27 06:07:00,28684.24,28720.09,28650.00,28720.09,1130.15185,41.697900,20.761369,40.221518,-2.714127,0,0
2022-05-27 06:08:00,28720.09,28727.83,28658.56,28665.68,89.60058,9.533654,18.770276,33.689268,-3.841976,1,0
2022-05-27 06:09:00,28665.67,28709.28,28656.70,28698.60,56.10624,32.400000,27.877184,40.034818,-1.882661,0,0
2022-05-27 06:10:00,28698.60,28721.33,28685.69,28715.47,30.07166,43.646667,28.526773,43.042793,0.888937,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2022-06-03 03:35:00,30492.54,30506.38,30492.54,30502.46,13.94515,60.572337,48.389566,49.289244,1.968542,0,0
2022-06-03 03:36:00,30502.46,30503.96,30502.45,30503.96,9.90027,63.222046,55.614438,49.963160,2.502973,0,0
2022-06-03 03:37:00,30503.95,30507.91,30503.95,30507.90,9.62998,70.181947,64.658776,51.775997,3.036572,0,0


In [179]:
class Signals:

  def __init__(self,df,lags):
    self.df = df
    self.lags = lags

  def gettrigger(self):
      dfnew = pd.DataFrame()
      for i in range(self.lags+1):
        mask = (self.df['%K'].shift(i) < 80) & (self.df['%D'].shift(i) <80)
        dfnew = dfnew.append(mask, ignore_index= True) 
        return dfnew.sum(axis=0)
  def decide(self):
    self.df['trigger'] = np.where(self.gettrigger(),1,0)
    self.df['Buy'] = np.where((self.df.trigger) & (self.df['%K'].between(20,80)) & (self.df['%D'].between(20,80)) & (self.df.rsi > 50) & (self.df.macd > 0),1,0)



In [183]:
def strategy(pair,qty, open_position=False):
  df= pulldata(pair,'1m','100')
  applytechnicals(df)
  inst = Signals(df,25)
  inst.decide()
  print(f'current Close is' + str(df.Close.iloc[-1]))
  if df.Buy.iloc[-1]:
    order = client.create_order(symbol = pair,side = 'BUY',type = 'MARKET', quantity = qty)
    print(order)
    buyprice= float(order['fills'][0]['price'])
    open_position = True
    while open_position:
      time.sleep(0.5)
      df = pulldata(pair,'1m','2')
      print(f'current Close' + str(df.Close.iloc[-1]))
      print(f'current Target' + str(buyprice * 1.005))
      print(f'current Stop' + str(buyprice*0.998))
      if df.Close[-1] <= buyprice*0.998 or df.Close[-1] >= buyprice*1.005:
        order = client.create_order(symbol = pair,side = 'SELL',type = 'MARKET', quantity = qty)
        print(order)
        break



In [185]:
while True:
  strategy('BTCUSDT',50)
  time.sleep(0.5)

current Close is30485.32
current Close is30485.32
current Close is30485.32
current Close is30485.31
current Close is30485.31
current Close is30483.51
current Close is30483.51
current Close is30481.64
current Close is30481.64
current Close is30480.9
current Close is30480.9
current Close is30480.9
current Close is30480.9
current Close is30480.9
current Close is30480.9
current Close is30480.9
current Close is30480.89
current Close is30480.89
current Close is30480.9
current Close is30480.9
current Close is30480.9
current Close is30480.9
current Close is30480.9
current Close is30480.9
current Close is30480.9
current Close is30480.9
current Close is30480.9
current Close is30480.9
current Close is30480.9
current Close is30480.89
current Close is30480.89
current Close is30480.9
current Close is30480.9
current Close is30475.17
current Close is30475.17
current Close is30475.17
current Close is30475.17
current Close is30475.17
current Close is30475.17
current Close is30475.16
current Close is3047

BinanceAPIException: ignored